In [1]:
import django, os, sys, time, resource, re, gc, shutil
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from django.db.models import Count, Avg
from multiprocess import Pool
from functools import partial
import numpy as np
from functools import partial
from scipy.sparse import coo_matrix, csr_matrix, find, tril
import networkx as nx

sys.path.append('/home/galm/software/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *
qid = 1457

def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

In [2]:
q = Query.objects.get(pk=qid)

mdocs = Doc.objects.filter(
    query=q,
    wosarticle__cr__isnull=False
)

referred_citations = Citation.objects.filter(
    referent__isnull=False,
    referent__UT__in=mdocs.values_list('UT',flat=True)
)
print(referred_citations.count())
rcs = referred_citations.values_list('id',flat=True)

42476


In [3]:
cdos = CDO.objects.filter(
    doc__query=q,
    citation__in=rcs
)
print(cdos.count())

m = mdocs.count()
m_dict = dict(zip(
    list(mdocs.values_list('UT',flat=True)),
    list(range(m))
))


rev_m_dict = dict(zip(
    list(range(m)),
    list(mdocs.values_list('UT',flat=True))
))

print("ROWIDS")
row_ids = list(cdos.values_list('doc__UT',flat=True))
rows = np.array([m_dict[x] for x in row_ids])

print("colids")
col_ids = list(cdos.values_list('citation__referent__UT',flat=True))
cols = np.array([m_dict[x] for x in col_ids])

print("data")
data = np.array([1]*cdos.count())

print("matrix")
Scoo = coo_matrix((data, (rows,cols)),shape=(m,m))

275151
ROWIDS
colids
data
matrix


In [4]:
S = Scoo.tocsr()
save_sparse_csr("../networks/1457_citations",S)


In [5]:
S > 2

<164837x164837 sparse matrix of type '<class 'numpy.bool_'>'
	with 9 stored elements in Compressed Sparse Row format>